# 1. 데이터 불러오기

In [34]:
## 필요한 패키지를 불러옵니다.
import os
import pandas as pd

In [35]:
## 경로설정을 해줍니다.
os.chdir(r'C:\Users\PC\Documents\강의최종본\data\앙상블')
## 위의 경로설정은 제 컴퓨터를 기준으로 입력한 것으로 각자 데이터를 저장한 위치에 맞게 바꿔주시기 바랍니다.

data = pd.read_csv("bank-additional-full.csv", sep = ';')
## 위의 실습 데이터는 러닝스푼즈 홈페이지에서도 다운받으실 수 있습니다.

In [36]:
## 불러온 데이터의 상위 5개 행을 봅니다.
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [37]:
## 카테고리형 변수를 수치화시키는 one hot encoding이란 방법론을 pd.get_dummies란 함수를 통해 실행시킵니다.
data = pd.get_dummies(data,columns=['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome'])

In [38]:
## 아이디를 생성합니다.
data['id']=range(len(data))

In [40]:
## data 중에서 30000개를 랜덤으로 추출해 train 데이터로 지정합니다.
train = data.sample(30000,replace=False,random_state=2020).reset_index().drop(['index'],axis=1)

In [1]:
## 추출되지 않은 나머지를 test 데이터로 지정합니다.
test = data.loc[ ~data['id'].isin(train['id']) ].reset_index().drop(['index'],axis=1)

In [1]:
## 학습에 활용할 input 변수를 지정합니다.
input_var = ['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed',
       'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'default_no', 'default_unknown', 'default_yes',
       'housing_no', 'housing_unknown', 'housing_yes', 'loan_no',
       'loan_unknown', 'loan_yes', 'contact_cellular', 'contact_telephone',
       'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep',
       'day_of_week_fri', 'day_of_week_mon', 'day_of_week_thu',
       'day_of_week_tue', 'day_of_week_wed', 'poutcome_failure',
       'poutcome_nonexistent', 'poutcome_success']

# 2. Xgboost 실습

<strong>특징</strong>

1. 해석이 어렵다.
2. 랜덤포레스트에 비해 빠르고 성능이 좋다(대체적으로).

<strong>파라미터</strong>

1. n_estimators : 몇 개의 의사결정나무를 만들 것인지
2. learning_rate : 얼마나 빠르게 학습할 것인지

<strong>Xgboost 학습시키기</strong>

In [78]:
## xgboost를 위한 패키지를 다운받습니다.
!pip install xgboost

  Using cached xgboost-1.0.2-py3-none-win_amd64.whl (24.6 MB)


In [79]:
## xgboost를 위한 패키지를 임포트합니다.
from xgboost import XGBClassifier

In [96]:
## xgboost 모델을 train 데이터에서 학습시킵니다.
xgb = XGBClassifier( n_estimators = 300, learning_rate = 0.1 )
xgb.fit(train[input_var], train['y'])

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=300, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [97]:
## 예측을 진행합니다.
predictions = xgb.predict(test[input_var])

In [98]:
## 정확도를 살펴봅니다.
(pd.Series(predictions)==test['y']).mean()

0.9141043975688238

<strong>최적의 n_estimators 찾기</strong>

In [104]:
for n in [100,200,300,400,500,600,700,800,900]:
    xgb = XGBClassifier( n_estimators = n, learning_rate = 0.05 )
    xgb.fit(train[input_var], train['y'])
    predictions = xgb.predict(test[input_var])
    print((pd.Series(predictions)==test['y']).mean())

0.9129424383267787
0.9140150160886664
0.9131212012870933
0.9146406864497676
0.9127636753664641
0.9115123346442617
0.9115123346442617
0.9111548087236324
0.9108866642831606


최적의 n_estimators는 400이다.

<strong>변수 중요도 뽑아내기(추후 변수중요도 강의에서 다룰 예정)</strong>

In [1]:
# feature_imp = xgb.feature_importances_

In [2]:
# imp_df = pd.DataFrame({'var':input_var,
#                        'imp':feature_imp})

# imp_df.sort_values(['imp'],ascending=False)